# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-12 06:52:12] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=39317, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=407063296, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-12 06:52:21] Attention backend not set. Use fa3 backend by default.
[2025-05-12 06:52:21] Init torch distributed begin.
[2025-05-12 06:52:21] Init torch distributed ends. mem usage=0.00 GB
[2025-05-12 06:52:21] Load weight begin. avail mem=38.82 GB


[2025-05-12 06:52:22] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:33<00:33, 33.32s/it]


...

[2025-05-12 06:53:33] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-12 06:53:34] INFO:     Started server process [2559970]
[2025-05-12 06:53:34] INFO:     Waiting for application startup.
[2025-05-12 06:53:34] INFO:     Application startup complete.
[2025-05-12 06:53:34] INFO:     Uvicorn running on http://0.0.0.0:39317 (Press CTRL+C to quit)


[2025-05-12 06:53:34] INFO:     127.0.0.1:35572 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 06:53:35] INFO:     127.0.0.1:35580 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 06:53:35] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:53:36] INFO:     127.0.0.1:35596 - "POST /generate HTTP/1.1" 200 OK
[2025-05-12 06:53:36] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 06:53:39] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:53:40] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 5.68, #queue-req: 0


[2025-05-12 06:53:41] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 63.37, #queue-req: 0


[2025-05-12 06:53:41] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, gen throughput (token/s): 63.77, #queue-req: 0


[2025-05-12 06:53:42] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, gen throughput (token/s): 65.07, #queue-req: 0


...

[2025-05-12 06:53:44] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, gen throughput (token/s): 64.80, #queue-req: 0


[2025-05-12 06:53:45] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, gen throughput (token/s): 65.88, #queue-req: 0


[2025-05-12 06:53:45] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, gen throughput (token/s): 65.20, #queue-req: 0


[2025-05-12 06:53:46] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, gen throughput (token/s): 64.66, #queue-req: 0


[2025-05-12 06:53:46] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, gen throughput (token/s): 64.35, #queue-req: 0
[2025-05-12 06:53:47] INFO:     127.0.0.1:35606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 06:53:47] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:53:47] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 53.09, #queue-req: 0


[2025-05-12 06:53:48] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 56.51, #queue-req: 0


[2025-05-12 06:53:48] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 64.57, #queue-req: 0


[2025-05-12 06:53:49] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 65.02, #queue-req: 0


[2025-05-12 06:53:50] Decode batch. #running-req: 1, #token: 212, token usage: 0.01, gen throughput (token/s): 65.75, #queue-req: 0


[2025-05-12 06:53:50] Decode batch. #running-req: 1, #token: 252, token usage: 0.01, gen throughput (token/s): 65.51, #queue-req: 0


[2025-05-12 06:53:51] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, gen throughput (token/s): 63.96, #queue-req: 0


[2025-05-12 06:53:52] Decode batch. #running-req: 1, #token: 332, token usage: 0.02, gen throughput (token/s): 60.06, #queue-req: 0


[2025-05-12 06:53:52] INFO:     127.0.0.1:35606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 06:53:52] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:53:52] Decode batch. #running-req: 1, #token: 42, token usage: 0.00, gen throughput (token/s): 54.01, #queue-req: 0


[2025-05-12 06:53:53] Decode batch. #running-req: 1, #token: 82, token usage: 0.00, gen throughput (token/s): 64.70, #queue-req: 0


[2025-05-12 06:53:54] Decode batch. #running-req: 1, #token: 122, token usage: 0.01, gen throughput (token/s): 62.44, #queue-req: 0


[2025-05-12 06:53:54] Decode batch. #running-req: 1, #token: 162, token usage: 0.01, gen throughput (token/s): 65.62, #queue-req: 0


...

[2025-05-12 06:53:55] Decode batch. #running-req: 1, #token: 242, token usage: 0.01, gen throughput (token/s): 61.84, #queue-req: 0


[2025-05-12 06:53:56] Decode batch. #running-req: 1, #token: 282, token usage: 0.01, gen throughput (token/s): 66.76, #queue-req: 0


[2025-05-12 06:53:57] Decode batch. #running-req: 1, #token: 322, token usage: 0.02, gen throughput (token/s): 65.06, #queue-req: 0


[2025-05-12 06:53:57] Decode batch. #running-req: 1, #token: 362, token usage: 0.02, gen throughput (token/s): 63.44, #queue-req: 0


[2025-05-12 06:53:58] Decode batch. #running-req: 1, #token: 402, token usage: 0.02, gen throughput (token/s): 66.91, #queue-req: 0
[2025-05-12 06:53:58] INFO:     127.0.0.1:35606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 06:53:58] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:53:59] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 61.45, #queue-req: 0


[2025-05-12 06:53:59] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 66.74, #queue-req: 0


[2025-05-12 06:54:00] INFO:     127.0.0.1:35606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 06:54:00] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:54:00] Decode batch. #running-req: 1, #token: 480, token usage: 0.02, gen throughput (token/s): 42.56, #queue-req: 0


[2025-05-12 06:54:01] Decode batch. #running-req: 1, #token: 520, token usage: 0.03, gen throughput (token/s): 65.46, #queue-req: 0


[2025-05-12 06:54:01] Decode batch. #running-req: 1, #token: 560, token usage: 0.03, gen throughput (token/s): 65.36, #queue-req: 0


[2025-05-12 06:54:02] Decode batch. #running-req: 1, #token: 600, token usage: 0.03, gen throughput (token/s): 65.33, #queue-req: 0


[2025-05-12 06:54:03] Decode batch. #running-req: 1, #token: 640, token usage: 0.03, gen throughput (token/s): 62.82, #queue-req: 0


[2025-05-12 06:54:03] Decode batch. #running-req: 1, #token: 680, token usage: 0.03, gen throughput (token/s): 63.95, #queue-req: 0


[2025-05-12 06:54:04] Decode batch. #running-req: 1, #token: 720, token usage: 0.04, gen throughput (token/s): 63.71, #queue-req: 0


[2025-05-12 06:54:04] INFO:     127.0.0.1:35606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-12 06:54:05] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:05] Decode batch. #running-req: 1, #token: 50, token usage: 0.00, gen throughput (token/s): 30.60, #queue-req: 0


[2025-05-12 06:54:06] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, gen throughput (token/s): 65.95, #queue-req: 0


[2025-05-12 06:54:06] Decode batch. #running-req: 1, #token: 130, token usage: 0.01, gen throughput (token/s): 66.72, #queue-req: 0


[2025-05-12 06:54:07] Decode batch. #running-req: 1, #token: 170, token usage: 0.01, gen throughput (token/s): 66.65, #queue-req: 0


...

[2025-05-12 06:54:08] Decode batch. #running-req: 1, #token: 250, token usage: 0.01, gen throughput (token/s): 64.94, #queue-req: 0


[2025-05-12 06:54:09] Decode batch. #running-req: 1, #token: 290, token usage: 0.01, gen throughput (token/s): 65.00, #queue-req: 0


[2025-05-12 06:54:09] Decode batch. #running-req: 1, #token: 330, token usage: 0.02, gen throughput (token/s): 65.93, #queue-req: 0


[2025-05-12 06:54:10] Decode batch. #running-req: 1, #token: 370, token usage: 0.02, gen throughput (token/s): 66.57, #queue-req: 0


[2025-05-12 06:54:10] INFO:     127.0.0.1:38936 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down.\n\nFirst, I need to identify the capital of France. I know that Paris is the capital, so that\'s straightforward. Now, I should find the most recent population data. I remember that the population of Paris has been growing, but I\'m not sure of the exact number. I think it\'s around 2 million, but I should verify that.\n\nWait, I should check the latest statistics to be accurate. Maybe I can recall that as of 2023, the population was approximately 2,150,000. That seems about right. I should make sure to include this number in the JSON.\n\nNext, I need to structure this information into a JSON format. JSON typically uses key-value pairs, so I\'ll create an object with keys like "city", "population", and "country". The city is Paris, the population is 2,150,000, and the 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-12 06:54:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:11] Decode batch. #running-req: 1, #token: 36, token usage: 0.00, gen throughput (token/s): 62.68, #queue-req: 0


[2025-05-12 06:54:11] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 66.14, #queue-req: 0


[2025-05-12 06:54:12] Decode batch. #running-req: 1, #token: 116, token usage: 0.01, gen throughput (token/s): 66.10, #queue-req: 0


[2025-05-12 06:54:12] Decode batch. #running-req: 1, #token: 156, token usage: 0.01, gen throughput (token/s): 65.08, #queue-req: 0


...

[2025-05-12 06:54:16] Decode batch. #running-req: 1, #token: 356, token usage: 0.02, gen throughput (token/s): 63.93, #queue-req: 0


[2025-05-12 06:54:16] Decode batch. #running-req: 1, #token: 396, token usage: 0.02, gen throughput (token/s): 66.71, #queue-req: 0


[2025-05-12 06:54:17] Decode batch. #running-req: 1, #token: 436, token usage: 0.02, gen throughput (token/s): 62.29, #queue-req: 0


[2025-05-12 06:54:17] Decode batch. #running-req: 1, #token: 476, token usage: 0.02, gen throughput (token/s): 65.38, #queue-req: 0


[2025-05-12 06:54:18] Decode batch. #running-req: 1, #token: 516, token usage: 0.03, gen throughput (token/s): 64.88, #queue-req: 0
[2025-05-12 06:54:18] INFO:     127.0.0.1:38940 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-12 06:54:18] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:54:18] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:54:18] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0


[2025-05-12 06:54:19] Decode batch. #running-req: 3, #token: 116, token usage: 0.01, gen throughput (token/s): 113.89, #queue-req: 0


[2025-05-12 06:54:20] Decode batch. #running-req: 3, #token: 236, token usage: 0.01, gen throughput (token/s): 188.62, #queue-req: 0


[2025-05-12 06:54:20] Decode batch. #running-req: 3, #token: 356, token usage: 0.02, gen throughput (token/s): 187.22, #queue-req: 0


[2025-05-12 06:54:21] Decode batch. #running-req: 3, #token: 476, token usage: 0.02, gen throughput (token/s): 180.47, #queue-req: 0


[2025-05-12 06:54:22] Decode batch. #running-req: 3, #token: 596, token usage: 0.03, gen throughput (token/s): 175.24, #queue-req: 0


[2025-05-12 06:54:22] Decode batch. #running-req: 1, #token: 246, token usage: 0.01, gen throughput (token/s): 86.55, #queue-req: 0


[2025-05-12 06:54:23] INFO:     127.0.0.1:49070 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-12 06:54:23] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:23] Decode batch. #running-req: 1, #token: 24, token usage: 0.00, gen throughput (token/s): 61.37, #queue-req: 0


[2025-05-12 06:54:24] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 59.88, #queue-req: 0


[2025-05-12 06:54:24] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, gen throughput (token/s): 65.15, #queue-req: 0


[2025-05-12 06:54:25] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, gen throughput (token/s): 65.53, #queue-req: 0


...

[2025-05-12 06:54:52] Decode batch. #running-req: 1, #token: 1904, token usage: 0.09, gen throughput (token/s): 64.86, #queue-req: 0


[2025-05-12 06:54:53] Decode batch. #running-req: 1, #token: 1944, token usage: 0.09, gen throughput (token/s): 64.28, #queue-req: 0


[2025-05-12 06:54:53] Decode batch. #running-req: 1, #token: 1984, token usage: 0.10, gen throughput (token/s): 64.24, #queue-req: 0


[2025-05-12 06:54:54] Decode batch. #running-req: 1, #token: 2024, token usage: 0.10, gen throughput (token/s): 64.99, #queue-req: 0


[2025-05-12 06:54:54] INFO:     127.0.0.1:49080 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-12 06:54:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:54:54] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 62.40, #queue-req: 0


[2025-05-12 06:54:55] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 65.79, #queue-req: 0


[2025-05-12 06:54:56] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 65.72, #queue-req: 0


[2025-05-12 06:54:56] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 62.33, #queue-req: 0


[2025-05-12 06:54:57] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 67.17, #queue-req: 0


...

[2025-05-12 06:55:01] Decode batch. #running-req: 1, #token: 433, token usage: 0.02, gen throughput (token/s): 65.21, #queue-req: 0


[2025-05-12 06:55:01] Decode batch. #running-req: 1, #token: 473, token usage: 0.02, gen throughput (token/s): 66.13, #queue-req: 0


[2025-05-12 06:55:02] Decode batch. #running-req: 1, #token: 513, token usage: 0.03, gen throughput (token/s): 66.76, #queue-req: 0


[2025-05-12 06:55:02] Decode batch. #running-req: 1, #token: 553, token usage: 0.03, gen throughput (token/s): 61.65, #queue-req: 0


[2025-05-12 06:55:03] INFO:     127.0.0.1:44036 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-05-12 06:55:03] Child process unexpectedly failed with an exit code 9. pid=2560484
[2025-05-12 06:55:03] Child process unexpectedly failed with an exit code 9. pid=2560550


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:03<00:03,  3.11s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:07<00:00,  3.63s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:07<00:00,  3.56s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
Okay, I need to find out information about the capital of France, which is Paris. Let me start by recalling what I know about Paris. 

First, I know that Paris is the largest city in France and its administrative capital. I remember hearing that it's also one of the most visited tourist destinations in the world. It's famous for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. 

I think Paris is located in the northern part of France. Maybe in the Île-de-France region? I believe it's situated between the Seine and the seine, which are the
Prompt: Give me the information of the capital of Germany.
Generated text: 
Okay, so I need to figure out the information about the capital of Germany. Hmm, I remember that the capital is a city in Germany, but I'm not exactly sure which one it is. Maybe it's Berlin? That sounds familiar. I think I've heard people talking about Berlin a lot, especia

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its population, economic status, cultural significance, and major landmarks. and include a summary of how London has evolved over the past 500 years.

Additionally, explain in detail the environmental challenges that London faces, including the impact of climate change and the effectiveness of its environmental policies, and discuss the potential solutions London can implement to address these challenges.

Also, if the UK were to leave the EU, would that have a negative impact on London's economy? Explain your reasoning.

Additionally, what is London's role in the global economy, including the number of multinational corporations headquartered in London, and the role of financial services?

Finally, discuss
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic status, cultural significance, and the Paris Agreement.

The user provided their ow

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Okay, so the user asked me to provide the information and population of the capital of France in JSON format. Hmm, let me break this down. First, I know that the capital of France is Paris. That's pretty straightforward. Now, I need to find the current population of Paris. I remember that populations can change over time, so I should make sure to use the most recent data available. I think the population of Paris is around 2 million people, but I'm not entirely sure about the exact number. Maybe it's a bit more or less? Let me double-check that. Oh, right, Paris has grown quite a bit over the years, especially with tourism and international students. I believe the current population is approximately 2,175,000. That seems about right. 

Wait, should I include the year this data was recorded? Probably not nec

In [19]:
llm.shutdown()